## 1: Create Client

In [60]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())

client: OpenAI = OpenAI()

## 2: Define Functions

In [62]:
import requests

#user_input = input("Enter your crypto name")

def get_crypto(token_name:str):
    
    url = f"https://api.coincap.io/v2/assets/{token_name.lower()}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return response.text
    

In [63]:
import json

def show_json(message,obj):
    display(json.loads(message, obj.model_dump_json()))

In [64]:
from openai.types.beta.assistant import Assistant

assistant: Assistant = client.beta.assistants.create(
    model="gpt-3.5-turbo-1106",
    tools=[{
        "type":"function",
        "function": {
            "name" : "get_crypto",
            "description": "get the priceUsd of token_name",
            "parameters":{
                "type":"object",
                "properties": {
                    "token_name": {"type":"string", "description": "The token price in usd e.g. 29.98"},
                    "priceUsd": {"type":"string"}     
                },
                "required":"token_name"
            }
        }
        
    }]
)

In [66]:
from openai.types.beta.thread import Thread
from openai.types.beta.threads.thread_message import ThreadMessage
from openai.types.beta.threads.run import Run

thread: Thread = client.beta.threads.create()

message: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Whats the usd price of bitcoin?"
)

run:Run = client.beta.threads.runs.create(
    assistant_id=assistant.id,
    thread_id=thread.id,
)
display(run)



Run(id='run_DN7NlUNwn4lBiZ066VjIezg8', assistant_id='asst_EqbpyXnZFCuMwXyZsilScwjy', cancelled_at=None, completed_at=None, created_at=1702690566, expires_at=1702691166, failed_at=None, file_ids=[], instructions=None, last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_syx4Il6Ec7jDXSSI0XyStXxd', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_crypto', parameters={'type': 'object', 'properties': {'token_name': {'type': 'string', 'description': 'The token price in usd e.g. 29.98'}}}, description='get the priceUsd of token_name'), type='function')])

In [ ]:
import time

available_function = {"get_crypto": get_crypto}

while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    runSteps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    print(runStatus.status, "......")
    
    if runStatus == "action_required":
        print(runStatus.status,'.....')
        